# Building Out Comparisons

Author: Jake Dumbauld <br>
Contact: jacobmilodumbauld@gmail.com<br>
Date: 3.15.22

In [1]:
#imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import time
import re
from IPython.display import clear_output

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import backend

Defining a Helper Function to Plot Confusion Matrix

In [2]:
def plot_CF(cf_matrix):
    """
    Takes an input confusion matrix as an NP array and returns a seaborn heatmap plot with labels.
    """
    group_names = ['True Neg','False Pos','False Neg','True Pos']

    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]

    group_percentages = ["{0:.2%}".format(value) for value in
                         cf_matrix.flatten()/np.sum(cf_matrix)]

    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
              zip(group_names,group_counts,group_percentages)]

    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

Defining Function to Train Models Iteratively

In [3]:
def model_n_trials(n, config_path, initial_learning_rate):
    """
    Trains a keras model from a given architecture specified at the `config_path` n times. Tracks train_accs, val_accs, test_accs, and generates a running confusion matrix across all trials.
    
    n: number of times to train the model
    config_path: filepath to a saved neural network
    initial_learning_rate: initial learning rate for the model to be used in the lr_schedule.
    """
    
    #loading in optimal model configuration from my model search
    best_config = keras.models.load_model(config_path).get_config()

    #defining learning rate
    lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
                  #initial learning rate copied from model search.
                  initial_learning_rate=initial_learning_rate,
                  decay_steps=1.0,
                  decay_rate=0.1
    )

    #defining a blank confusion matrix to keep a running total
    final_cf = np.zeros((2,2), np.int32)
    #defining a lists for other metrics of interest
    train_accs = []
    val_accs = []
    test_accs = []

    for i in range(0,n):
        print("Trial " + str(i+1) + "/30 Begin")
        model = keras.Sequential.from_config(best_config)

        model.compile(
            #Optimizer
            optimizer = keras.optimizers.Adam(learning_rate=lr_schedule),
            #Loss
            loss=keras.losses.BinaryCrossentropy(),
            #Metrics
            metrics=[keras.metrics.BinaryAccuracy()]
        )
        #training model with an early stopping callback
        es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=(es_callback))

        #appending training accuracy
        train_accs.append(history.history["binary_accuracy"][-1])
        #appending validation accuracy
        val_accs.append(history.history["val_binary_accuracy"][-1])
        
        print("Evaluating Model " + str(i+1))
        
        #appending test accuracy
        test_accs.append(model.evaluate(X_test,y_test)[1])
        #adding running totals to the overall confusion matrix
        y_pred= (model.predict(X_test) > 0.5).astype("int32")
        cf_matrix = tf.math.confusion_matrix(labels=y_test, predictions=y_pred).numpy()
        final_cf += cf_matrix

        #necessary to avoid memory issues from iteratively training kears models that I read about online.
        backend.clear_session()
        #clearing output text after each trial to avoid screen vomit
        clear_output(wait=True)
    
    return final_cf, train_accs, val_accs, test_accs

Importantly - NOT setting random seed here. I want to ensure that there is some randomness in the training process to demonstrate the robustness of my output models.

Defining the Model From the Best HPs found in Model Search

In [ ]:
#loading in and reshaping data to train model
X = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/MFCCs_withPatient.npy', allow_pickle=True)
y = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/target_array.npy', allow_pickle=True)

X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.3)

#training model 30 times, randomly initializing each time. 
CNN_MFCC_w_cf, CNN_MFCC_w_train_accs, CNN_MFCC_w_val_accs, CNN_MFCC_w_test_accs = model_n_trials(30, 
                                                                                                 '/Users/jmd/Documents/BOOTCAMP/Capstone/neural_nets/CNN_MFCCs_with_patient',
                                                                                                 0.00020681360209481224)

Trial 13/30 Begin
Epoch 1/100
46/46 [==============================] - 7s 150ms/step - loss: 0.6575 - binary_accuracy: 0.7388 - val_loss: 0.4707 - val_binary_accuracy: 0.7959
Epoch 2/100
46/46 [==============================] - 7s 142ms/step - loss: 0.4801 - binary_accuracy: 0.7909 - val_loss: 0.4570 - val_binary_accuracy: 0.8054
Epoch 3/100
46/46 [==============================] - 7s 148ms/step - loss: 0.4596 - binary_accuracy: 0.8089 - val_loss: 0.4529 - val_binary_accuracy: 0.8149
Epoch 4/100
46/46 [==============================] - 7s 147ms/step - loss: 0.4464 - binary_accuracy: 0.8103 - val_loss: 0.4552 - val_binary_accuracy: 0.8228
Epoch 5/100
46/46 [==============================] - 7s 149ms/step - loss: 0.4323 - binary_accuracy: 0.8390 - val_loss: 0.4440 - val_binary_accuracy: 0.8149
Epoch 6/100
46/46 [==============================] - 7s 147ms/step - loss: 0.4300 - binary_accuracy: 0.8284 - val_loss: 0.4494 - val_binary_accuracy: 0.8101
Epoch 7/100
46/46 [=====================

In [ ]:
#dividing by thirty to generate averages for the 30 trials run above. 
plot_CF(CNN_MFCC_final_cf/30)